# Training/Test Data H/0 F/1


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import cv2 
import numpy as np
import os

In [ ]:
dossier=os.listdir("/content/drive/MyDrive/Adience/Adience_Homme_Femme/Femme")
Femme=np.zeros((len(dossier),64,64,3))
Labels_F=[]
for i in range(len(dossier)):
  nom_image='/content/drive/MyDrive/Adience/Adience_Homme_Femme/Femme/'+dossier[i]
  image = cv2.resize((cv2.imread(nom_image)), (64,64))
  Femme[i]=image
  Labels_F.append(0)

In [ ]:
dossier1=os.listdir("/content/drive/MyDrive/Adience/Adience_Homme_Femme/Homme")
Homme=np.zeros((len(dossier1),64,64,3))
Labels_H=[]
for i in range(len(dossier1)):
  nom_image='/content/drive/MyDrive/Adience/Adience_Homme_Femme/Homme/'+dossier1[i]
  image = cv2.resize((cv2.imread(nom_image)), (64,64))
  Homme[i]=image
  Labels_H.append(1)

In [ ]:
len(Labels_F)

10107

In [ ]:
len(Labels_H)

8378

In [ ]:
def train (x):
  return int(x*0.8)

In [ ]:
images_train_Homme=Homme[:train(len(Homme))]
images_test_Homme=Homme[train(len(Homme)):]
labels_train_Homme=Labels_H[:train(len(Labels_H))]
labels_test_Homme=Labels_H[train(len(Labels_H)):]

images_train_Femme=Femme[:train(len(Femme))]
images_test_Femme=Femme[train(len(Femme)):]
labels_train_Femme=Labels_F[:train(len(Labels_F))]
labels_test_Femme=Labels_F[train(len(Labels_F)):]

In [ ]:
X_train=np.concatenate((images_train_Homme,images_train_Femme),axis=0)
Y_train=np.concatenate((labels_train_Homme,labels_train_Femme),axis=0)
X_test=np.concatenate((images_test_Homme,images_test_Femme),axis=0)
Y_test=np.concatenate((labels_test_Homme,labels_test_Femme),axis=0)

In [ ]:

print(len(X_train))
print(len(Y_train))
print(len(X_test))
print(len(Y_test))

14787
14787
3698
3698


In [ ]:
pip install keras-facenet


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from keras_facenet import FaceNet


In [ ]:
model = FaceNet()


In [ ]:
def get_embedding(model, face):
    # transfer face into one sample (3 dimension to 4 dimension):ajoute une dimension supplémentaire à l'image du visage
    #afin qu'elle puisse être transmise à travers le modèle. (4 dim: 1:taille de lot , 3: H,L, canaux de l'image)
    sample = np.expand_dims(face, axis=0) 
    # make prediction to get embedding:passe le sample tableau à travers la embeddings méthode du model
    #pour générer l'incorporation de l'image du visage.
    yhat = model.embeddings(sample)
    return yhat[0]#renvoie l'intégration sous la forme d'un tableau NumPy unidimensionnel.


In [ ]:
# convert each face in the train set into embedding
emdTrainX = list()
for face in X_train:
    emd = get_embedding(model,face)
    emdTrainX.append(emd)
    
emdTrainX = np.asarray(emdTrainX)
print(emdTrainX.shape)



Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
1/1 [==============================] - 0s 94ms/step
(14787, 512)


In [ ]:
# convert each face in the test set into embedding
emdTestX = list()
for face in X_test:
    emd = get_embedding(model, face)
    emdTestX.append(emd)
emdTestX = np.asarray(emdTestX)
print(emdTestX.shape)

1/1 [==============================] - 0s 93ms/step
(3698, 512)


In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC

print("Dataset: train=%d, test=%d" % (emdTrainX.shape[0], emdTestX.shape[0]))
# normalize input vectors
in_encoder = Normalizer()
emdTrainX_norm = in_encoder.transform(emdTrainX)
emdTestX_norm = in_encoder.transform(emdTestX)



Dataset: train=14787, test=3698


In [ ]:
# label encode targets
#un encodage des étiquettes (labels)
out_encoder = LabelEncoder()#transforme des étiquettes de texte (ou de catégories) en entiers.
out_encoder.fit(Y_train)#calcule les correspondances entre les étiquettes et les entiers
trainy_enc = out_encoder.transform(Y_train)#applique ces correspondances à la variable d'entrée et renvoie les entiers correspondants.


In [ ]:
testy_enc = out_encoder.transform(Y_test)

In [ ]:
# fit model
model = SVC(kernel='rbf',degree=3, gamma=1,probability=True,C=100 )
model.fit(emdTrainX_norm, trainy_enc)
# predict
#prédiction des classes sur data train & test à l'aide d'un modèle de classification
yhat_train = model.predict(emdTrainX_norm)
yhat_test = model.predict(emdTestX_norm)


# score:calcule la précision de la prédiction pour chaque ensemble.
score_train = accuracy_score(trainy_enc, yhat_train)
score_test = accuracy_score(testy_enc, yhat_test)
# summarize
print('Accuracy: train=%.3f, test=%.3f' % (score_train*100, score_test*100))

Accuracy: train=99.912, test=85.506


In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import ConfusionMatrixDisplay
res = confusion_matrix(testy_enc, yhat_test)
for i in range(0,len(res)):
  print(res[i])
print("Confusion Matrix")
print(res)
print(f"Test Set: {len(Y_test)}")
print(f"Accuracy = {score_test*100} %")
print(classification_report(testy_enc, yhat_test))

[1745  277]
[ 259 1417]
Confusion Matrix
[[1745  277]
 [ 259 1417]]
Test Set: 3698
Accuracy = 85.50567874526772 %
              precision    recall  f1-score   support

           0       0.87      0.86      0.87      2022
           1       0.84      0.85      0.84      1676

    accuracy                           0.86      3698
   macro avg       0.85      0.85      0.85      3698
weighted avg       0.86      0.86      0.86      3698

